In [45]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

MODEL = "gpt-4o-mini"
openai = OpenAI()

In [9]:
class Website:
    url: str
    title: str
    body: str
    links = List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


ed = Website("https://edwarddonner.com")

In [13]:
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

In [14]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [15]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [22]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages = [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)},
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)


get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'API page', 'url': 'https://www.anthropic.com/api'},
  {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}]}

In [25]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'API page', 'url': 'https://www.anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-p

In [51]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [32]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]
    return user_prompt

print(get_brochure_user_prompt("Anthropic", "https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://www.anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}]}
You are looking at a company called: Anthropic
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now availab

In [34]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'pricing page', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://www.anthropic.com/api'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic Company Brochure

Welcome to **Anthropic**, your trusted partner in the development of safe and reliable artificial intelligence solutions.

---

## About Us

### Overview

Anthropic is an **AI safety and research company** based in San Francisco. Our mission revolves around building robust, interpretable, and steerable AI systems that ensure beneficial outcomes for society. At Anthropic, we believe that the development of AI technology is a double-edged sword, and we are committed to illuminating the path toward safety in this transformative space.

### Our Purpose

We aim to make AI systems people can rely on, responding to the vast potential of AI's impact on the world. Our work includes frontier research, safety methodologies, and the deployment of AI-driven products that foster societal advancement.

---

## Our Product: Claude

### Meet Claude

Claude is our signature AI model that helps businesses streamline their operations while enhancing productivity through collaborative tools. Claude is designed not just for efficiency but also for safety, ensuring that all applications prioritize user well-being.

**Key Features:**
- Easy integration through API for customized solutions
- Enhancements in creative and analytical tasks
- Real-time collaboration across diverse teams

---

## Company Culture

### Our Values

At Anthropic, we embrace a culture of **trust, collaboration, and mission-driven work**. Here’s how we define our workplace ethos:

1. **Here for the Mission**: We prioritize building AI systems that promote flourishing for humanity.
2. **Unusually High Trust**: We foster an environment where kindness and honesty thrive.
3. **One Big Team**: Collaboration across disciplines is central to our operations.
4. **Do the Simple Thing That Works**: We focus on pragmatic solutions that yield real results.

### Employee Well-Being

We believe our employees are our greatest asset. That's why we provide a comprehensive suite of health benefits, parental leave, flexible paid time off, and wellness stipends, alongside competitive salaries and equity packages.

---

## Join the Team

### Careers at Anthropic

Anthropic is looking for passionate individuals eager to contribute to the future of safe AI development. We offer a range of roles in engineering, research, policy, and operations, with a flexible, collaborative environment conducive to both personal and professional growth.

**What We Offer**:
- A mission that motivates and challenges
- Flexible working arrangements and remote options
- Opportunities for professional growth and innovation

### How to Apply

We encourage candidates from diverse backgrounds to apply, whether or not they range from traditional machine learning experience. Our interview process is designed to assess unique strengths and contributions to our mission.

---

## Our Clients

We cater to businesses, nonprofits, and civil organizations, assisting them in harnessing the power of AI effectively and safely. Our products empower teams across various functions including Engineering, Marketing, and Support, revolutionizing how they operate.

---

### Get in Touch

For more information about our products, partnerships, or career opportunities, please visit our website or contact us through our social media platforms.

- [Twitter](https://twitter.com/Anthropic)
- [LinkedIn](https://www.linkedin.com/company/anthropic)
- [YouTube](https://www.youtube.com/Anthropic)

---

**Anthropic** aims to lead the charge in ensuring that AI technology drives positive change for humanity. Join us in building a safe AI future!

---

In [52]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'claude page', 'url': 'https://www.anthropic.com/claude'}]}


# Welcome to Anthropic: AI Safety with a Side of Humor! 🐜🤖

---

**Hey there, Future Innovator!**  
Are you looking for a company where safety and creativity go hand in hand, all while keeping a smile on your face? Welcome to **Anthropic**, your friendly neighborhood AI safety and research company based in fabulous San Francisco! 🌉

---

### Who's in Charge of All This Fun? 🤔
Meet **Claude**! No, not the guy in your office who steals your lunch—**Claude** is our state-of-the-art AI model, helping teams work better, faster, and with more creativity than you can shake a stick at! Also, we might have a minor obsession with naming things after poets, so look out for Claude 3.5 Sonnet and Claude 3.5 Haiku, on the shelves soon in your local AI supermarket! 📜✨

---

### Our Purpose: Making AI Work for the Good of Humanity (and Snacks) 🍕🎉
At Anthropic, we're all about building AI systems you can **RELY** on. Kind of like that one reliable friend who always has snacks and good advice ready to go! We prioritize safety and treat it like a science (because it is, you see). Not to mention, we're looking out for the opportunities AI can offer as well! 

---

### The Team: A Star-Studded Cast of Characters 🌟
Our interdisciplinary crew includes:

- **Researchers & Engineers**: Crafting AI systems that would make even the most sophisticated robots jealous. 
- **Policy Experts**: Sorting through the maze of regulations while making sense of them (kind of like a legal Sudoku).
- **Business Leaders**: Ensuring we stay on track without losing our way to the nearest taco truck! 🌮

---

### Life at Anthropic: Like a Technological Playground (But with Safety Nets) 🤹‍♂️
Here's what you'll find when you join our team:

- **Health & Wellness**: Because you can’t save the world on an empty stomach! We offer comprehensive health plans and mental health support. 🥗💊
  
- **Flexible PTO**: Time off for good behavior or spontaneous taco truck excursions—**no questions asked!**

- **Daily Feasts**: We have meals and snacks available in the office to keep our creative juices flowing! 🍫

- **Team Spirit**: We promote a culture of **trust** and **honesty**—we assume good intentions, disagree with kindness, and always try to do the simple thing that works!

---

### Join Us: Your Future Awaits! 🚀
Whether you’re an AI whiz or have never coded a thing in your life, we love diverse experiences! Not a PhD holder? No problem! Come as you are! We’ve got roles for everyone (except maybe for that obscure hobby you have of collecting Victorian-themed snow globes—unless it’s really your thing). 

Our mission is responsibility, safety, and a race to the top on **AI safety!** This means if you help us succeed, we won't just offer you a salary, we'll offer you a chance to change the world (while looking fabulous)!

---

### Potential Customers: Say Hello to Claude! 🙌
Join global innovators who use **Claude** for everything from coding help to creative brainstorming! “Claude helped us transform weeks of research into minutes! And honestly, we still had time to binge-watch our favorite shows!” - *Some actual happy customer*

---

### Ready to Take the Plunge? 🏊‍♀️
Let’s build the future of AI together! Whether you’re looking for your next adventure in tech, want to partner with us, or need AI assistance that keeps you smiling, Anthropic is the place for you.

---

**Join us! Because who doesn’t love a little AI-powered magic?** ✨

--- 

*For more information, please visit our website, and don’t forget to bring your sense of humor along with your application!* 😄

In [50]:
stream_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
Hugging Face is a leading community-driven platform that is shaping the future of artificial intelligence (AI) and machine learning (ML). We provide a collaborative environment where developers, researchers, and enterprises can contribute to and enhance a wide variety of models, datasets, and applications. Our mission is to democratize AI and ML, making them accessible to everyone.

## Our Offering
Hugging Face serves as a comprehensive hub for:
- **400k+ Models:** A diverse range of state-of-the-art machine learning models ready for public use.
- **100k+ Datasets:** Access to an extensive collection of datasets for text, image, video, audio, and even 3D tasks.
- **150k+ Applications:** Browse innovative applications powered by the latest AI technologies.

### Key Features
- **Open Source Tools:** We actively contribute to the foundation of ML tooling with numerous projects including Transformers, Diffusers, and Tokenizers, empowering the community to build better AI solutions.
- **Compute Solutions:** Hugging Face offers tailored computing infrastructures, starting at just $0.60/hour, optimized for rapid deployment of ML applications.
- **Enterprise Solutions:** Our enterprise-grade platform provides dedicated support, security features, and advanced tools for organizations looking to leverage AI effectively.

## Customers
Our platform is trusted by over **50,000 organizations**, including industry giants such as:
- Microsoft
- Google
- Amazon Web Services
- Grammarly
- Meta
- Intel

## Company Culture
At Hugging Face, we pride ourselves on fostering a vibrant and inclusive community. Our culture emphasizes:
- **Collaboration:** We believe in the power of community and work closely with our users to drive innovation.
- **Transparency:** Open-source values guide our operations, ensuring that we share knowledge and resources freely.
- **Growth:** We encourage continuous learning and development, enabling our team and community to grow with the industry.

## Careers
Join our mission to revolutionize AI! We are continuously looking for passionate individuals to help us build tools that advance the frontiers of machine learning. Whether you are an engineer, researcher, or marketer, we offer various roles to fit your skills and aspirations. Explore our current openings [here](#).

## Connect With Us
- **Social Media:** Follow us on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), and [Discord](https://discord.gg/huggingface) to stay updated with the latest in AI.
- **Documentation & Resources:** Visit our [Docs](https://huggingface.co/docs) and [Blog](https://huggingface.co/blog) for tutorials, guides, and community stories.

Join the Hugging Face community and help build the future of AI!